In [31]:
import pyranges as pr
import pandas as pd
import numpy as np
import h5py 
import os
import cerberus
import swan_vis as swan

## Add novelty of annotated transcriptome

In [66]:
fname = '/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/cerberus/test.h5'
ca = cerberus.read(fname)

In [67]:
ca.t_map.source.unique()

array(['lapa', 'v29'], dtype=object)

In [68]:
annot_source = 'lapa'
ref_sources = ['v29', 'v40']

In [69]:
def get_feats():
    return ['tss', 'ic', 'tes']

In [70]:
for feat in get_feats():
    
    # pick the right df to look at
    if feat == 'tss':
        df = ca.tss
    elif feat == 'ic':
        df = ca.ic
    elif feat == 'tes':
        df = ca.tes
    
    # restrict to our reference sources; take the union
    inds = []
    for source in ref_sources:
        inds += df.loc[df.source.str.contains(source)].index.tolist()
    inds = list(set(inds))
    ref_ids = df.loc[inds].Name.tolist()
    
    # add a novelty column to the transcript map
    nov_col = '{}_novelty'.format(feat)
    id_col = '{}_id'.format(feat)
    ca.t_map[nov_col] = 'Novel'
    ca.t_map.loc[ca.t_map[id_col].isin(ref_ids), nov_col] = 'Known'

In [71]:
ca.t_map.loc[ca.t_map.transcript_id=='ENSG00000000003[1,1,5]']

,original_transcript_id,ic,ic_id,tss_id,tss,tes_id,tes,gene_id,gene_name,original_transcript_name,transcript_triplet,transcript_id,transcript_name,tss_first_sd_issue,tes_last_sa_issue,source,tss_novelty,ic_novelty,tes_novelty
319996,ENCODEHT000604802#0,1.0,NaN,ENSG00000000003_1,1.0,ENSG00000000003_5,5.0,ENSG00000000003,TSPAN6,ENCODEHT000604802,"[1,1,5]","ENSG00000000003[1,1,5]","TSPAN6[1,1,5]",False,False,lapa,Known,Novel,Known
320003,ENCODEHT000605067#0,1.0,NaN,ENSG00000000003_1,1.0,ENSG00000000003_5,5.0,ENSG00000000003,TSPAN6,ENCODEHT000605067,"[1,1,5]","ENSG00000000003[1,1,5]","TSPAN6[1,1,5]",False,False,lapa,Known,Novel,Known
321921,ENST00000373020.8,1.0,ENSG00000000003_1,ENSG00000000003_1,1.0,ENSG00000000003_5,5.0,ENSG00000000003,TSPAN6,TSPAN6-201,"[1,1,5]","ENSG00000000003[1,1,5]","TSPAN6[1,1,5]",False,False,lapa,Known,Known,Known
321925,ENST00000373020.8#3,1.0,ENSG00000000003_1,ENSG00000000003_1,1.0,ENSG00000000003_5,5.0,ENSG00000000003,TSPAN6,TSPAN6-201,"[1,1,5]","ENSG00000000003[1,1,5]","TSPAN6[1,1,5]",False,False,lapa,Known,Known,Known
203329,ENST00000373020.8,1.0,ENSG00000000003_1,ENSG00000000003_1,1.0,ENSG00000000003_5,5.0,ENSG00000000003,TSPAN6,TSPAN6-201,"[1,1,5]","ENSG00000000003[1,1,5]","TSPAN6[1,1,5]",False,False,v29,Known,Known,Known


In [33]:
sg = swan.read('/Users/fairliereese/Documents/programming/mortazavi_lab/data/rnawg/lr_bulk/cerberus/swan.p')

Read in graph from /Users/fairliereese/Documents/programming/mortazavi_lab/data/rnawg/lr_bulk/cerberus/swan.p


In [72]:
ca.ic.head()

,Chromosome,Strand,Coordinates,Name,source,gene_id,ic
0,chr1,+,,ENSG00000004487_18,"v40,v29,lapa",ENSG00000004487,18
1,chr1,+,,ENSG00000033122_5,"v40,v29,lapa",ENSG00000033122,5
2,chr1,+,,ENSG00000049246_6,"v40,v29,lapa",ENSG00000049246,6
3,chr1,+,,ENSG00000077157_8,"v40,v29,lapa",ENSG00000077157,8
4,chr1,+,,ENSG00000099260_3,"v40,v29,lapa",ENSG00000099260,3


In [38]:
# reconcile novelty 
temp = ca.t_map.copy(deep=True)
# temp = temp.loc[temp.source == annot_source]
temp = temp[['transcript_id']+['{}_novelty'.format(feat) for feat in get_feats()]]
temp.rename({'transcript_id': 'tid'}, axis=1, inplace=True)
temp.drop_duplicates(inplace=True)
sg.t_df = sg.t_df.merge(temp, how='left', left_index=True, right_on='tid')

In [56]:
temp = sg.t_df[['tid', 'ic_novelty', 'novelty']].copy(deep=True)
temp.groupby(['ic_novelty', 'novelty'], dropna=False).count().reset_index()

,ic_novelty,novelty,tid
0,Known,ISM,829
1,Known,Known,241123
2,Known,NIC,2723
3,Known,NNC,650
4,Novel,ISM,2865
5,Novel,Known,3917
6,Novel,NIC,29015
7,Novel,NNC,8891
8,Novel,Undefined,817
9,NaN,ISM,23994


In [65]:
ca.t_map.loc[ca.t_map.transcript_id == 'ENSG00000000003[1,1,5]']

,original_transcript_id,ic,ic_id,tss_id,tss,tes_id,tes,gene_id,gene_name,original_transcript_name,transcript_triplet,transcript_id,transcript_name,tss_first_sd_issue,tes_last_sa_issue,source,ic_novelty,tss_novelty,tes_novelty
319996,ENCODEHT000604802#0,1.0,NaN,ENSG00000000003_1,1.0,ENSG00000000003_5,5.0,ENSG00000000003,TSPAN6,ENCODEHT000604802,"[1,1,5]","ENSG00000000003[1,1,5]","TSPAN6[1,1,5]",False,False,lapa,Novel,Known,Known
320003,ENCODEHT000605067#0,1.0,NaN,ENSG00000000003_1,1.0,ENSG00000000003_5,5.0,ENSG00000000003,TSPAN6,ENCODEHT000605067,"[1,1,5]","ENSG00000000003[1,1,5]","TSPAN6[1,1,5]",False,False,lapa,Novel,Known,Known
321921,ENST00000373020.8,1.0,ENSG00000000003_1,ENSG00000000003_1,1.0,ENSG00000000003_5,5.0,ENSG00000000003,TSPAN6,TSPAN6-201,"[1,1,5]","ENSG00000000003[1,1,5]","TSPAN6[1,1,5]",False,False,lapa,Known,Known,Known
321925,ENST00000373020.8#3,1.0,ENSG00000000003_1,ENSG00000000003_1,1.0,ENSG00000000003_5,5.0,ENSG00000000003,TSPAN6,TSPAN6-201,"[1,1,5]","ENSG00000000003[1,1,5]","TSPAN6[1,1,5]",False,False,lapa,Known,Known,Known
203329,ENST00000373020.8,1.0,ENSG00000000003_1,ENSG00000000003_1,1.0,ENSG00000000003_5,5.0,ENSG00000000003,TSPAN6,TSPAN6-201,"[1,1,5]","ENSG00000000003[1,1,5]","TSPAN6[1,1,5]",False,False,v29,Known,Known,Known


In [62]:
temp.loc[(temp.ic_novelty == 'Novel')&(temp.novelty=='Known')]

,tid,ic_novelty,novelty
319996.0,"ENSG00000000003[1,1,5]",Novel,Known
319997.0,"ENSG00000000003[1,1,7]",Novel,Known
319998.0,"ENSG00000000003[1,1,8]",Novel,Known
22531.0,"ENSG00000000938[1,1,2]",Novel,Known
259682.0,"ENSG00000001167[1,1,1]",Novel,Known
...,...,...,...
81748.0,"ENSG00000279965[1,1,1]",Novel,Known
261017.0,"ENSG00000280707[1,1,2]",Novel,Known
196601.0,"ENSG00000282826[1,1,2]",Novel,Known
111782.0,"ENSG00000283221[1,1,1]",Novel,Known


In [64]:
ca.ic.loc[ca.ic.Name == 'ENSG00000001167_1']

,Chromosome,Strand,Coordinates,Name,source,gene_id,ic
197210,chr6,+,41073084-41079028-41079164-41080810-41080897-4...,ENSG00000001167_1,"v40,v29,lapa",ENSG00000001167,1


In [50]:
ca.t_map.loc[ca.t_map.transcript_id == 'ENSG00000000003[1,1,9]']

,original_transcript_id,ic,ic_id,tss_id,tss,tes_id,tes,gene_id,gene_name,original_transcript_name,transcript_triplet,transcript_id,transcript_name,tss_first_sd_issue,tes_last_sa_issue,source,ic_novelty,tss_novelty,tes_novelty


In [52]:
ca.tes.loc[ca.tes.Name == 'ENSG00000000003_9']

,Chromosome,Start,End,Strand,Name,source,gene_id,tes
219504,chrX,100629123,100629132,-,ENSG00000000003_9,"pas,polya_atlas",ENSG00000000003,9


## Make sure that appending to cerberus object is working correctly

In [4]:
fname = '/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/cerberus/test.h5'
ca = cerberus.read(fname)

In [6]:
ca.t_map.source.unique()

array(['lapa', 'v29'], dtype=object)

## where are the no coordinate ics going?

In [2]:
h5 = '~/mortazavi_lab/data/rnawg/lr_bulk/cerberus/cerberus_ref.h5'
ic_ref = '~/mortazavi_lab/data/rnawg/lr_bulk/cerberus/agg_ics.tsv'

In [5]:
# df = read_ic_ref(ic_ref, add_gid=True, add_num=True) # pass
ic, tss, tes, tss_map, tes_map, m = read_h5(h5) # pass

In [7]:
ic.head()

,Chromosome,Strand,Coordinates,Name,source,gene_id,ic
0,chr1,+,,ENSG00000004487_18,"v40,v29,lapa",ENSG00000004487,18
1,chr1,+,,ENSG00000033122_5,"v40,v29,lapa",ENSG00000033122,5
2,chr1,+,,ENSG00000049246_6,"v40,v29,lapa",ENSG00000049246,6
3,chr1,+,,ENSG00000077157_8,"v40,v29,lapa",ENSG00000077157,8
4,chr1,+,,ENSG00000099260_3,"v40,v29,lapa",ENSG00000099260,3


## fix issue with agg_2_ends w/ duplicated regions

In [2]:
def make_end_df(c,s,st,e,n, source,mode):
    df = pd.DataFrame()
    cols = ['Chromosome', 'Strand', 'Start', 'End', 'Name']
    var = [c,s,st,e,n]
    for col, var in zip(cols, var):
        if type(var) == list:
            df[col] = var

    # add source
    df['source'] = source

    df = format_end_df(df)

    # get end # and gene id
    if any(df.Name.isnull()):
        df['gene_id'] = np.nan
        df[mode] = np.nan
    else:
        df['gene_id'] = df.Name.str.split('_', expand=True)[0]
        df[mode] = df.Name.str.split('_', expand=True)[1]

    # get arbitrary unique ids
    df['id'] = [i for i in range(len(df.index))]

    return df#

def format_end_df(df):
    sort_cols = ['Chromosome', 'Start', 'End', 'Strand']
    df = df.sort_values(by=sort_cols)
    order = ['Chromosome', 'Start', 'End', 'Strand', 'Name', 'source']
    order = [o for o in order if o in df.columns]
    df = df[order]
    df.reset_index(drop=True, inplace=True)
    return df

def test_agg_2_ends_1(print_dfs=True):
    """
    Test agg_2_ends w/ and w/o end adding
    """

    def get_test(mode='tss'):
        # example has
        # - entries that overlap
        # - entries that don't overlap but are within a certain distance
        # - entries that are unique to either
        # - entries that overlap but aren't using the same gene_id / strand (these will be
        #      equivalent situtations b/c the gene ids for things on different strands will always
        #      differ
        # - entries in bed2 that are non continuous but overlap the same
        #      entry in bed1
        # - entries in bed1 that the same entry in bed2 maps to that are all the same gene
        #       bed1: gene3_1, gene3_2
        #       bed2: gene3_1
        # - entries in bed1 that the same entry in bed2 maps to that are NOT from the same gene;
        #    need to create new end
        #       bed1: gene3_1, gene3_2
        #       bed2: gene4_1
        # - entry in bed1 that multiple entries in bed2 overlap
        #      

        slack = 20

        n = 6
        c = ['1' for i in range(n)]
        s = ['+' for i in range(n)]
        st = [1, 200, 100, 300, 700, 760]
        e = [15, 250, 110, 340, 750, 770] 
        n = ['gene1_1', 'gene1_2', 'gene1_3', 'gene1_4', 'gene3_1', 'gene3_2']
        source = 'v1'
        bed1 = make_end_df(c,s,st,e,n, source, mode)
        bed1 = pr.PyRanges(bed1)

        n = 7
        c = ['1' for i in range(n)]
        s = ['+' for i in range(n)]
        st = [5, 20, 120, 500, 200, 750, 750]
        e = [10, 25, 140, 550, 250, 775, 775]
        n = ['gene1_1', 'gene1_4', 'gene1_2', 'gene1_3', 'gene2_1', 'gene3_1', 'gene4_1']
        source = 'v2'
        bed2 = make_end_df(c,s,st,e,n, source, mode)
        bed2 = pr.PyRanges(bed2)

        return bed1, bed2

    def get_ctrl(add=True):

        mode = 'tss'

        n = 9
        c = ['1' for i in range(n)]
        s = ['+' for i in range(n)]
        st = [1,200,100,300,700,760, 200,500,750]
        e = [15,250,110,340,750,770, 250,550,775]
        n = ['gene1_1', 'gene1_2', 'gene1_3', 'gene1_4', 'gene3_1', 'gene3_2', 'gene2_1', 'gene1_5', 'gene4_1']
        source = ['v1,v2','v1','v1,v2','v1','v1,v2','v1,v2', 'v2','v2','v2']
        df = make_end_df(c,s,st,e,n, source,mode)

        # convert a few dtypes
        df['Strand'] = df['Strand'].astype('category')
        df['Chromosome'] = df['Chromosome'].astype('category')

        # if we're not adding new ends
        if not add:
            df = df.loc[df.source != 'v2']

        # remove unnecessary columns
        df.drop(['gene_id', 'id', mode], axis=1, inplace=True)

        # fix ids
        df.reset_index(drop=True, inplace=True)

        return df

    def get_ctrl_source_map(add=True):
        # n = 9
        # c = ['1' for i in range(n)]
        # s = ['+' for i in range(n)]
        # st = [1, 200, 100, 300,  5, 20, 120, 500, 200]
        # e = [15, 250, 110, 340,  10, 25, 140, 550, 250]
        # n = ['gene1_1', 'gene1_2', 'gene1_3', 'gene1_4',
        #      'gene1_1', 'gene1_1', 'gene1_3', 'gene1_5', 'gene2_1']
        # source = ['v1' for i in range(4)]+['v2' for i in range(5)]
        # mode = 'tss'
        # df = make_end_df(c,s,st,e,n, source,mode)
        # df = df.loc[df.source == 'v2']
        # df.drop(['gene_id', mode, 'id'], axis=1, inplace=True)

        n = 7
        c = ['1' for i in range(n)]
        s = ['+' for i in range(n)]
        st = [5, 20, 120, 500, 200, 750, 750]
        e = [10, 25, 140, 550, 250, 775, 775]
        n = ['gene1_1', 'gene1_1', 'gene1_3', 'gene1_5', 'gene2_1', 'gene3_1', 'gene3_2']
        source = 'v2'
        mode = 'tss'
        df = make_end_df(c,s,st,e,n, source,mode)
        df = df.loc[df.source == 'v2']
        df.drop(['gene_id', mode, 'id'], axis=1, inplace=True)
        df['Strand'] = df['Strand'].astype('category')
        df['Chromosome'] = df['Chromosome'].astype('category')

        if not add:
            new_names = ['gene2_1', 'gene1_5']
            new_inds = df.loc[df.Name.isin(new_names)].index
            df.loc[new_inds, 'Name'] = np.nan

        return df

    tests = [True, False]
    for add_ends in tests:

        slack = 20
        mode = 'tss'
        sort_cols = ['Chromosome', 'Strand', 'gene_id', 'Start', 'End']
        order = ['Chromosome', 'Start', 'End', 'Strand', 'Name',
                 'gene_id', 'source', mode]
        bed1, bed2 = get_test()

        df, m_source = agg_2_ends(bed1, bed2,
                        strand=True,
                        gid=True,
                        slack=slack,
                        add_ends=add_ends,
                        mode=mode)

        test = format_end_df(df)
        test_m = format_end_df(m_source)
        ctrl = get_ctrl(add=add_ends)
        ctrl_m = get_ctrl_source_map(add=add_ends)

        if print_dfs:
            print('test')
            print(test)
            print(test.index)
            print(test.dtypes)
            print('ctrl')
            print(ctrl)
            print(ctrl.index)
            print(ctrl.dtypes)
            print('test source map')
            print(test_m)
            print(test_m.index)
            print(test_m.dtypes)
            print('ctrl source map')
            print(ctrl_m)
            print(ctrl_m.index)
            print(ctrl_m.dtypes)

        pd.testing.assert_frame_equal(ctrl, test, check_like=True)
        assert len(ctrl.index) == len(test.index)
    
        pd.testing.assert_frame_equal(ctrl_m, test_m, check_like=True)
        assert len(ctrl_m.index) == len(test_m.index)

# def test_agg_2_ends_2(print_dfs=True):
#     """
#     Test agg_2_ends w/ and w/o end adding
#     """

#     def get_test(mode='tss'):

#         # adding a bed file that doesn't have strand or gid info
#         # entries in bed2 that are duplicated based on
#         # - strandedness
#         # - gene id
#         # entries in bed2 where both of them overlap the same region
#         # in bed1

#         slack = 20

#         n = 6
#         c = ['1' for i in range(n)]
#         s = ['+' for i in range(n)]
#         s[-1] = '-'
#         st = [1, 200, 100, 300, 260, 260]
#         e = [15, 250, 110, 340, 290, 290]
#         n = ['gene1_1', 'gene1_2', 'gene1_3', 'gene1_4', 'gene2_1', 'gene3_1']
#         source = 'v1'
#         bed1 = make_end_df(c,s,st,e,n, source, mode)
#         bed1 = pr.PyRanges(bed1)

#         n = 5
#         c = ['1' for i in range(n)]
#         s = [np.nan for i in range(n)]
#         st = [5, 11, 120, 500, 200]
#         e = [10, 21, 140, 550, 250]
#         n = [np.nan for i in range(n)]
#         source = 'v2'
#         bed2 = make_end_df(c,s,st,e,n, source, mode)
#         bed2 = pr.PyRanges(bed2)

#         return bed1, bed2

#     def get_ctrl(add=True):

#         mode = 'tss'

#         n = 6
#         c = ['1' for i in range(n)]
#         s = ['+' for i in range(n)]
#         s[-1] = '-'
#         st = [1,200,100,300, 260,260]
#         e = [15,250,110,340, 290,290]
#         n = ['gene1_1', 'gene1_2', 'gene1_3', 'gene1_4', 'gene2_1', 'gene3_1']
#         source = ['v1,v2','v1,v2','v1,v2','v1', 'v1,v2', 'v1,v2']
#         df = make_end_df(c,s,st,e,n, source,mode)

#         # convert a few dtypes
#         df['Strand'] = df['Strand'].astype('category')
#         df['Chromosome'] = df['Chromosome'].astype('category')

#         # if we're not adding new ends
#         if not add:
#             df = df.loc[df.source != 'v2']

#         # remove unnecessary columns
#         df.drop(['gene_id', 'id', mode], axis=1, inplace=True)

#         # fix ids
#         df.reset_index(drop=True, inplace=True)

#         return df

#     def get_ctrl_source_map(add=False):

#         n = 7
#         c = ['1' for i in range(n)]
#         s = ['+' for i in range(n)]
#         s[-2] = '-'
#         s[-1] = np.nan
#         st = [5, 11, 120, 200, 200, 200, 500]
#         e = [10, 21, 140, 250, 250, 250, 550]
#         n = ['gene1_1', 'gene1_1', 'gene1_3', 'gene1_2', 'gene2_1', 'gene3_1', np.nan]
#         source = 'v2'
#         mode = 'tss'
#         df = make_end_df(c,s,st,e,n, source,mode)
#         df = df.loc[df.source == 'v2']
#         df.drop(['gene_id', mode, 'id'], axis=1, inplace=True)
#         df['Strand'] = df['Strand'].astype('category')
#         df['Chromosome'] = df['Chromosome'].astype('category')

#         return df

#     strand = False
#     gid = False
#     add_ends = False

#     slack = 20
#     mode = 'tss'
#     sort_cols = ['Chromosome', 'Strand', 'gene_id', 'Start', 'End']
#     order = ['Chromosome', 'Start', 'End', 'Strand', 'Name',
#              'gene_id', 'source', mode]
#     bed1, bed2 = get_test()
#     df, m_source = agg_2_ends(bed1, bed2,
#                     strand=strand,
#                     gid=gid,
#                     slack=slack,
#                     add_ends=add_ends,
#                     mode=mode)
#     test = format_end_df(df)
#     test_m = format_end_df(m_source)
#     ctrl = get_ctrl(add=add_ends)
#     ctrl_m = get_ctrl_source_map()

#     if print_dfs:
#         print('test')
#         print(test)
#         print(test.index)
#         print(test.dtypes)
#         print('ctrl')
#         print(ctrl)
#         print(ctrl.index)
#         print(ctrl.dtypes)
#         print('test source map')
#         print(test_m)
#         print(test_m.index)
#         print(test_m.dtypes)
#         print('ctrl source map')
#         print(ctrl_m)
#         print(ctrl_m.index)
#         print(ctrl_m.dtypes)

#     pd.testing.assert_frame_equal(ctrl, test, check_like=True)
#     assert len(ctrl.index) == len(test.index)

#     pd.testing.assert_frame_equal(ctrl_m, test_m, check_like=True)
#     assert len(ctrl_m.index) == len(test_m.index)

In [ ]:
test_agg_2_ends_1(print_dfs=True)

> /Users/fairliereese/Documents/programming/mortazavi_lab/bin/cerberus/cerberus/cerberus.py(428)agg_2_ends()
    426 
    427     # situation 1: ends match across the datasets in coord and gene id
--> 428     if gid:
    429         temp = temp_joined.loc[temp_joined.gene_id == temp_joined.gene_id_new].copy(deep=True)
    430     else:



ipdb>  temp_joined.head()


  Chromosome  Start  End Strand     Name source gene_id tss  id  Start_new  \
0          1      1   15      +  gene1_1     v1   gene1   1   0        5.0   
1          1      1   15      +  gene1_1     v1   gene1   1   0       20.0   
2          1    100  110      +  gene1_3     v1   gene1   3   1      120.0   
3          1    200  250      +  gene1_2     v1   gene1   2   2      200.0   
8          1    300  340      +  gene1_4     v1   gene1   4   3        NaN   

   End_new Strand_new Name_new source_new gene_id_new tss_new  id_new  
0       10          +  gene1_1         v2       gene1       1       0  
1       25          +  gene1_4         v2       gene1       4       1  
2      140          +  gene1_2         v2       gene1       2       2  
3      250          +  gene2_1         v2       gene2       1       3  
8       -1          +       -1         -1          -1      -1      -1  


ipdb>  temp_joined


  Chromosome  Start  End Strand     Name source gene_id tss  id  Start_new  \
0          1      1   15      +  gene1_1     v1   gene1   1   0        5.0   
1          1      1   15      +  gene1_1     v1   gene1   1   0       20.0   
2          1    100  110      +  gene1_3     v1   gene1   3   1      120.0   
3          1    200  250      +  gene1_2     v1   gene1   2   2      200.0   
8          1    300  340      +  gene1_4     v1   gene1   4   3        NaN   
4          1    700  750      +  gene3_1     v1   gene3   1   4      750.0   
5          1    700  750      +  gene3_1     v1   gene3   1   4      750.0   
6          1    760  770      +  gene3_2     v1   gene3   2   5      750.0   
7          1    760  770      +  gene3_2     v1   gene3   2   5      750.0   

   End_new Strand_new Name_new source_new gene_id_new tss_new  id_new  
0       10          +  gene1_1         v2       gene1       1       0  
1       25          +  gene1_4         v2       gene1       4       1  
2  

...

> /Users/fairliereese/Documents/programming/mortazavi_lab/bin/cerberus/cerberus/cerberus.py(485)agg_2_ends()
    483                                (temp_joined.gene_id_new!='-1')].copy(deep=True)
    484         # pdb.set_trace()
--> 485         temp.drop(drop_cols, axis=1, inplace=True)
    486         temp.rename(m, axis=1, inplace=True)
    487         new_df = pd.concat([new_df, temp])



ipdb>  n


> /Users/fairliereese/Documents/programming/mortazavi_lab/bin/cerberus/cerberus/cerberus.py(486)agg_2_ends()
    484         # pdb.set_trace()
    485         temp.drop(drop_cols, axis=1, inplace=True)
--> 486         temp.rename(m, axis=1, inplace=True)
    487         new_df = pd.concat([new_df, temp])
    488 



ipdb>  nn


*** NameError: name 'nn' is not defined


## write h5 ref from input beds and ics tsv

In [70]:
ic = '/Users/fairliereese/Documents/programming/mortazavi_lab/data/rnawg/lr_bulk/cerberus/temp/talon_ic.tsv'
tes = '/Users/fairliereese/Documents/programming/mortazavi_lab/data/rnawg/lr_bulk/cerberus/test_tes.bed'
tss = '/Users/fairliereese/Documents/programming/mortazavi_lab/data/rnawg/lr_bulk/cerberus/test_tss.bed'
tss_map = '/Users/fairliereese/Documents/programming/mortazavi_lab/data/rnawg/lr_bulk/cerberus/test_tes_source_map.bed'
tes_map = '/Users/fairliereese/Documents/programming/mortazavi_lab/data/rnawg/lr_bulk/cerberus/test_tss_source_map.bed'

# df = read_ic_ref(ic)
# df.head()

write_reference(tss, tes, ic, 'test.h5')

# df = read_cerberus_ends(tss, mode='tss')
# df

df = read_cerberus_source_map(tss_map)
df.head()

,Chromosome,Start,End,Strand,source,Name
0,chr1,169804296,169804436,+,v40,ENSG00000000460_2
1,chr1,169807786,169807887,+,v40,ENSG00000000460_4
2,chr1,169821668,169821769,+,v40,ENSG00000000460_5
3,chr1,169852986,169853135,+,v40,ENSG00000000460_3
4,chr1,169854029,169854130,+,v40,ENSG00000000460_1


In [32]:
import pandas as pd
import numpy as np

df = pd.DataFrame()
n = 5
test_1 = [np.nan for i in range(n)]
test_1[-1] = 'b'
df['test_1'] = test_1
df['test_1'] = df.test_1.astype('category')
df['test_2'] = ['a' for i in range(n)]

o = 'test.h5'
df.to_hdf(o, 'test', mode='w', format='table')

df = pd.read_hdf(o, key='test')

In [33]:
df.head()

,test_1,test_2
0,NaN,a
1,NaN,a
2,NaN,a
3,NaN,a
4,b,a
